# Scale and bond-length statistics
Compute bond-length distributions and RMS before/after scaling for `data/train`.


In [1]:
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

from scripts.dataloader import HiCStructureDataset
from scripts.preprocess import center_batch, scale_batch, COORD_IDX, MASK_IDX

device = torch.device("cpu")

dataset = HiCStructureDataset(root_dir="data/train")
struct_paths = [s for _, s in dataset.samples]
len(struct_paths)


Found 37500 samples in dataset.


37500

In [2]:
coord_idx = torch.tensor(COORD_IDX, device=device)
mask_idx = torch.tensor(MASK_IDX, device=device)

def flatten_coords(struct_batch: torch.Tensor):
    coords = struct_batch[..., coord_idx]
    masks = struct_batch[..., mask_idx]
    coords_3d = coords.view(struct_batch.shape[0], -1, 3)
    valid = masks.repeat_interleave(3, dim=-1).view(struct_batch.shape[0], -1, 3)[..., 0] > 0.5
    return coords_3d, valid

def bond_lengths(struct_batch: torch.Tensor):
    coords_3d, valid = flatten_coords(struct_batch)
    diffs = coords_3d[:, 1:, :] - coords_3d[:, :-1, :]
    valid_pairs = valid[:, 1:] & valid[:, :-1]
    lengths = torch.linalg.norm(diffs, dim=-1)
    return lengths[valid_pairs]

def rms_from_lengths(lengths: torch.Tensor) -> float:
    return torch.sqrt(torch.mean(lengths ** 2)).item()


In [ ]:
center_lengths = []
scaled_lengths = []

with torch.no_grad():
    for struct_path in tqdm(struct_paths, desc="Loading structures"):
        struct = dataset._load_structure_seq(struct_path).unsqueeze(0).to(device)
        centered, _ = center_batch(struct)
        l_center = bond_lengths(centered)
        if l_center.numel() > 0:
            center_lengths.append(l_center.cpu())
        scaled, _ = scale_batch(centered)
        l_scaled = bond_lengths(scaled)
        if l_scaled.numel() > 0:
            scaled_lengths.append(l_scaled.cpu())

center_lengths = torch.cat(center_lengths)
scaled_lengths = torch.cat(scaled_lengths)

center_rms = rms_from_lengths(center_lengths)
scaled_rms = rms_from_lengths(scaled_lengths)

(center_lengths.numel(), scaled_lengths.numel(), center_rms, scaled_rms)


Loading structures:  23%|██▎       | 8567/37500 [12:49<41:43, 11.56it/s]  

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(center_lengths.numpy(), bins=120, color="#69b3a2", alpha=0.8)
ax.axvline(center_rms, color="crimson", linestyle="--", label=f"RMS={center_rms:.3f}")
ax.set_xlabel("Bond length (centered)")
ax.set_ylabel("Count")
ax.set_title("Bond-length distribution (centered)")
ax.legend()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax.hist(scaled_lengths.numpy(), bins=120, color="#5170d7", alpha=0.8)
ax.axvline(scaled_rms, color="orange", linestyle="--", label=f"RMS={scaled_rms:.3f}")
ax.set_xlabel("Bond length (center+scaled)")
ax.set_ylabel("Count")
ax.set_title("Bond-length distribution (scaled)")
ax.legend()
plt.show()
